# Object Detection (Часть 1)

> 🚀 В этой практике нам понадобятся: `numpy==1.26.4, matplotlib==3.8.4, scikit-learn==1.4.2, scikit-image==0.23.2, torch==2.1.2, torchvision==0.16.2, opencv-python==4.9.0.80` 

> 🚀 Установить вы их можете с помощью команды: `%pip install numpy==1.26.4 matplotlib==3.8.4 scikit-learn==1.4.2 scikit-image==0.23.2 torch==2.1.2 torchvision==0.16.2 opencv-python==4.9.0.80` 


## Содержание

* [Данные](#Данные)
* [Вычислительное железо](#Вычислительное-железо)
* [Загрузка сети](#Загрузка-сети)
* [Наконец-то, Object Detection](#Наконец-то,-Object-Detection)
* [Задание](#Задание)
* [Вопросики](#Вопросики)
* [Полезные ссылки](#Полезные-ссылки)


Всем привет! 

Сегодня мы будем разрушать мифы о том, что машинное обучение - это всегда обязательно обучение модели на триллионе данных в течение всей жизни. 

Основная мысль простая: **НЕ НАДО ИЗОБРЕТАТЬ ВЕЛОСИПЕД**. 

<p align="center"><img src="https://raw.githubusercontent.com/AleksDevEdu/ml_edu/master/assets/bicycle.jpg" width=400/></p>

Существует ряд задач, для которых уже есть решение, и прежде чем изобретать своё, есть смысл проверить как работают существующие решения, может быть они уже решают вашу задачу с достаточным качеством. 

"Существующие решения" в данном контексте - это предобученные модели, часто они уже встроены в библиотеки, либо их веса доступны в open-source. 

В этот раз мы с вами посмотрим, как можно осуществлять детекцию и распознавание различных объектов на изображениях. Мы будем использовать модели: 
* Faster R-CNN (ResNet) - более точная, но медленная
* Faster R-CNN (MobileNet) - работает быстрее, но менее точная 
* RetinaNet - неплохой баланс между скоростью работы и точностью.

> Не пугайтесь непонятных названий, это всего ли названия сетей. Каждая сеть/архитектура обычно имеет собственное название, чтобы их можно было различать. 

> 🤓 Вы можете подробнее прочитать про эти сети (их архитектуру) самостоятельно: [Faster R-CNN](https://arxiv.org/pdf/1506.01497.pdf), [R-CNN | Fast R-Cnn | Faster R-CNN](https://vbystricky.github.io/2017/06/rcnn_etc.html), [RetinaNet](https://arxiv.org/pdf/1708.02002.pdf), [RetinaNet (harb)](https://habr.com/ru/post/510560/) 

In [ ]:
# Настройки для визуализации
# Если используется темная тема - лучше текст сделать белым
import os 
import time

import cv2 
import torch 
import matplotlib.pyplot as plt 
import numpy as np
import random
TEXT_COLOR = "black"

# Зафиксируем состояние случайных чисел
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

## Данные

Мы с вами будем использовать датасет: [COCO (Common Objects in Context) Dataset](https://cocodataset.org/#home). Точнее, не сам датасет, а сверточные нейросети, обученные на этом датасете.

<p align="center"><img src="https://raw.githubusercontent.com/AleksDevEdu/ml_edu/master/assets/coco.png" width=700/></p>

Изначально этот датасет включал в себя 91 класс объектов, которые вы можете увидеть в повседневной жизни - животные, машины, люди, диваны и т.д. Но в [документации](https://tech.amikelive.com/node-718/what-object-categories-labels-are-in-coco-dataset/) написано, что в релизы 2014 и 2017 годов вошли только 80 классов (что всё ещё довольно много, а без распознавания расчёски мы в этот раз как-нибудь обойдёмся). 

> Там же в документации есть ссылки на GitHub и общие инструкции для получения данных. 

Ну, данные и данные, скажете вы, что нам с ними делать, куда засунуть? 
И это хороший вопрос! 

На самом деле, так как мы будем иcпользовать уже предобученные модели, то делить наши данные на выборки, обучать модель на обучающей выборки, подбирать её гиперпараметры - **ничего этого делать не нужно**! То есть всё, что у нас есть - это уже как бы тестовая выборка. Божественно, же? 

Однако, для наших высших целей, нам всё же нужно немного пощупать данные. А именно, нам нужно вытащить данные с соответствием ID-шников и их классов. Например, модель нам вернёт ID = 2, нам нужно понимать какому лейблу (классу) этот ID соответствует. 

Подготовленный список лейблов можно найти [здесь](https://pytorch.org/vision/stable/models.html#object-detection-instance-segmentation-and-person-keypoint-detection). Давай просто скопируем этот список в наш ноутбук. 

In [ ]:
COCO_INSTANCE_CATEGORY_NAMES = [
    "__background__", "person", "bicycle", "car", "motorcycle", "airplane", "bus",
    "train", "truck", "boat", "traffic light", "fire hydrant", "N/A", "stop sign",
    "parking meter", "bench", "bird", "cat", "dog", "horse", "sheep", "cow",
    "elephant", "bear", "zebra", "giraffe", "N/A", "backpack", "umbrella", "N/A", "N/A",
    "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball",
    "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket",
    "bottle", "N/A", "wine glass", "cup", "fork", "knife", "spoon", "bowl",
    "banana", "apple", "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza",
    "donut", "cake", "chair", "couch", "potted plant", "bed", "N/A", "dining table",
    "N/A", "N/A", "toilet", "N/A", "tv", "laptop", "mouse", "remote", "keyboard", "cell phone",
    "microwave", "oven", "toaster", "sink", "refrigerator", "N/A", "book",
    "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"
]

Таким образом, индекс лейбла соответствует его ID, и если модель возвращает 2, то нам нужно просто взять элемент по индексу 2. 

Ну и на последок, давайте сгенерируем массив цветов для каждого класса (лейбла). Мы ведь хотим выделять на изображении объекты, пусть их рамки будут разных цветов. Мы будем работать с цветными картинками, так что логично генерировать 3 канала сразу. 

In [ ]:
rng = np.random.default_rng(seed=RANDOM_SEED)
roi_colors = rng.uniform(0, 255, size=(len(COCO_INSTANCE_CATEGORY_NAMES), 3))

roi_colors.shape

Шикарно! Теперь у нас есть список с маппингом, который нам понадобится, чтобы расшифровать показания сети, и массив цветов для рамок объектов. Пора браться за самое вкусненькое!

## Вычислительное железо

Сначала давайте определимся с доступным железом, на котором будут происходить все вычисления.

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

DEVICE

* Если у вас есть видеокарта NVIDIA и все драйвера установлены правильно, то вы увидите - `device(type="cuda")`.
* Если видеокарты нет или косяк с драйверами, то увидите - `device(type="cpu")`, в таком случае сеть будет выполняться (inference) на процессоре.

## Загрузка сети

Для начала давайте создадим словарь со всеми сетками, которые нам интересны (этот шаг не обязательный, но так просто будет удобнее и меньше опечаток в процессе).

> Загружать готовые сети будем из фреймворка [torchvision](https://pytorch.org/vision/stable/index.html). А вот и ссылка на [раздел с сетями](https://pytorch.org/vision/stable/models.html#object-detection-instance-segmentation-and-person-keypoint-detection).

In [ ]:
from torchvision.models import detection 

MODELS = {
	"frcnn-resnet": detection.fasterrcnn_resnet50_fpn,
	"frcnn-mobilenet": detection.fasterrcnn_mobilenet_v3_large_320_fpn,
	"retinanet": detection.retinanet_resnet50_fpn
}

Давайте возьмём первую сетку, как мы помним, она самая точная из 3-х, но при этом и самая медленная. 

In [ ]:
# загружаем предобученную модель (pretrained, pretrained_backbone) 
model = MODELS["frcnn-resnet"](weights=True, progress=True, weights_backbone=True)
# отправляем её на наше вычислительное устройство
model = model.to(DEVICE)
# переключаем модель в evaluation-режим
model.eval()

В выводе вы можете увидеть структуру сети, обратите внимание, что последний слой на вход принимает 91 фичей - это как раз те исходные COCO классы. 

Но у нас же всего 80?! Что делать то? 

Есть два варианта: 

1. Ничего не делать, расслабиться и получать удовольствие. Если сетка выплюнет класс, которого нет в нашем словаре, то мы его просто пропустим (предварительно запомнив, что такие риски есть - тут уже нужно оценить на сколько вам важно определять расчёску или дверь на изображении).

2. Вручную добавить в наш словарь недостающие классы (если вы посмотрите внимательно, то в id-значениях есть пропуски, их можно заполнить, воспользовавшись документацией). 

Предлагаем пока не дёргаться и использовать тот словарь, который у нас уже есть. 

## Наконец-то, Object Detection

Логично, что чтобы что-то детектировать, нам нужна входная картинка или видео. Давайте загрузим таковую c использованием URL.

> Если вы захотите использовать локальную картинку, то можно использовать функцию из OpenCV `cv2.imread(img_fpath)`

In [ ]:
from skimage import io

# Используем scikit-image фреймворк, чтобы загрузить по URL. Он загружает изображения в формате RGB
url = "https://raw.githubusercontent.com/AleksDevEdu/ml_edu/master/data/45_od_test.jpg"
original_image = io.imread(url,plugin="matplotlib")

# Отображаем картинку и наслаждаемся 
plt.figure(figsize=[10, 12])
plt.imshow(original_image)
plt.title("Original Image")
plt.show()

Важный шаг - это преобразовать исходную картинку в формат, который сеть может понять. 

In [ ]:
# Создаём копию исходной картинки, чтобы её не менять
image = original_image.copy()

# Изменяем порядок следования размерностей
#   сейчас у нас следующий порядок: [height, width, channels] (aka [rows, columns, depth])
#   приводим его к [channels, height, width]
print(f"Original Ordering: {image.shape}")
image_chw = image.transpose((2, 0, 1))
print(f"Converted Ordering: {image_chw.shape}")

In [ ]:
# Добавляем новую размерность для размера батча
image_exp = np.expand_dims(image_chw, axis=0)
print(f"Image shape with batch: {image_exp.shape}")

In [ ]:
# Конвертируем значения пикселей из диапазона [0, 255] в диапазон [0, 1]
print(f"Original Pixels Limits: [{image_exp.min()}, {image_exp.max()}]")
image_norm = image_exp / 255.0
print(f"Converted Pixels Limits: [{image_norm.min()}, {image_norm.max()}]")

In [ ]:
# Преобразуем матрицу в тензор 
print(f"Original Image Type: {type(image_norm)}")
image_t = torch.FloatTensor(image_norm)
print(f"Converted Image Type: {type(image_t)}")

In [ ]:
# Переводим тензор на наше вычислительное устройство 
image_t = image_t.to(DEVICE)

start_ts = time.time()
# Запускаем сеть и получаем детектированные области и предсказание к ним 
detections = model(image_t)[0]
print(f"Detections - done! Time: {(time.time() - start_ts):.3f} sec.")

In [ ]:
detections

Здорово! Вот мы и получили предсказания. Согласитесь кода не так уж и много, умные люди уже всё сделали за нас. 

Но давайте посмотрим, а что вообще за предсказания мы получили, вдруг там вообще всё неправильно. Удобнее всего это сделать просмотрев картинку и отобразив на ней детектированные области с распознанными классами.

In [ ]:
# Создадим ещё одну копию исходной картинки, чтобы на ней рисовать рамки и классы 
draw_image = original_image.copy()
# Создадим словарь, в который скопируем только валидные данные 
valid_detection = dict()

# Итерируемся по всем предсказаниям 
for i in range(0, len(detections["boxes"])):
	# Достаём степень уверенности сети в своём предсказании 
	confidence = detections["scores"][i]
	
	# Используем только "уверенные" предсказания, то есть сеть уверена в них больше, чем на 70% 
	if confidence > 0.7:
		# Достаём значение ID лейбла (по которому мы найдём категории в нашем списке маппингов)
		idx = int(detections["labels"][i])
		
		# Достаём координаны боксов детектированной области 
		box = detections["boxes"][i].detach().cpu().numpy()
		(startX, startY, endX, endY) = box.astype("int")

		# Для отладки выведем информацию о предсказаниях
		pred_info = f"{COCO_INSTANCE_CATEGORY_NAMES[idx]}: {(confidence * 100):.2f}"
		print(pred_info)
		
		# Сохраним в наш словарь валидную информацию 
		valid_detection[COCO_INSTANCE_CATEGORY_NAMES[idx]] = box.astype("int")

		# Нарисуем детектированные боксы на изображении
		cv2.rectangle(
			draw_image, 
			(startX, startY), 
			(endX, endY),
			roi_colors[idx], 
			4
		)
		# Добавим название предсказанного класса рядом с боксом (для удобства)
		y = startY - 40 if startY - 40 > 40 else startY + 40
		cv2.putText(draw_image, pred_info, (startX, y),
			cv2.FONT_HERSHEY_SIMPLEX, 2, roi_colors[idx], 3)

# Отобразим картинку с дополнительной информацией
plt.figure(figsize=[10, 12])
plt.imshow(draw_image)
plt.show()

Успех! Сеть распознала всё правильно, ни один котик в результате эксперимента не пострадал! 

Теперь давайте посмотрим, что у нас хранится в валидных данных.

In [ ]:
valid_detection

В словаре в значениях мы сохранили координаты прямоугольников (box). Это боксы можно обозвать ещё ROI = Region of Intrest. Давайте попробуем отобразить только ROI из картинки. 

In [ ]:
fig, axs = plt.subplots(1, len(valid_detection), figsize=[10, 12])

# Итерируемся по данным в словаре
for ind, (label, box) in enumerate(valid_detection.items()):
    # Получаем координаты ROI
    (startX, startY, endX, endY) = valid_detection[label]
    # Картинка - это по сути матрица, поэтому слайсинг здесь работает как обычно
    roi_image = original_image[startY:endY, startX: endX]

    axs[ind].imshow(roi_image)
    axs[ind].set_title(label, fontsize=16)

plt.show()

Это ещё один способ, как визуально можно оценить качество работы модели. 

Но как же ещё оценить качество работы количественно? Мы с вами уже рассматривали много разных метрик (accuracy, recall, precision, f1, и т.д.). Можно ли их как-то применить для оценки распознавания? 

Ответ - можно. Но есть нюансы! Эти метрики используются для оценки задач классификации. Следовательно, надо привести задачу распознавания к задаче классификации. То есть для каждой тестовой картинки нужно подготовить "разметку". Например, на нашем изображении есть человек и кот, ставим в эти лейблы 1-ки, все остальные лейблы получают 0-ли. И дальше уже сравниваем с показаниями сети (можно добавить порог по уверенности, а можно смотреть вообще все "предсказания" сети).

## Задание

1. Попробуйте загрузить свою картинку и проверить как на ней отработает модель 
2. Попробуйте использовать оставшиеся две модели и сравнить их качество работы (визуально) и скорость на разных изображениях.
3. Попробуйте написать класс, который упрощает работу с задачей детектирования: создаешь класс, передаешь картинку, а в результате выдается список bbox с лейблами.

## Вопросики 

1. Зачем переводить картинку в Tensor?
2. Можно ли использовать сеть Faster R-CNN Resnet для детектирования и распознавания на видео в **реальном времени**?
3. Зачем проводить оценку работы предобученной сети? 
4. Можно ли обучать предобученную сеть? 
5. Можно ли оценить качество работы детектора людей этих сетей на совсем другом [датасете](http://host.robots.ox.ac.uk/pascal/VOC/)? Если можно, то что нужно для этого сделать? 

## Полезные ссылки
* [PyTorch object detection with pre-trained networks](https://www.pyimagesearch.com/2021/08/02/pytorch-object-detection-with-pre-trained-networks/)
